# ML Проект: Кластеризация новостей с автоматическим тегированием

## Постановка задачи

**Цель**: Создать систему автоматической кластеризации новостей и присвоения тематических тегов.

**Задачи**:
1. Кластеризация слов из всех новостей для формирования словаря тегов (центры кластеров слов = теги)
2. Кластеризация самих новостей по тематикам
3. Автоматическое присвоение тегов новостям на основе кластеров
4. Классификация новостей по найденным кластерам

**Метрики**:
- Для кластеризации: Silhouette Score, Davies-Bouldin Index
- Для классификации: Accuracy, F1-score, Precision, Recall
- Дополнительно: Confusion Matrix для анализа ошибок

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. Установка и импорт библиотек

In [ ]:
# Установка необходимых библиотек
!pip install -q pymorphy3 gensim nltk wordcloud scikit-learn pandas numpy matplotlib seaborn --break-system-packages

In [ ]:
import sqlite3
import json
import re
import warnings
from pathlib import Path
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Preprocessing
import pymorphy3
from nltk.corpus import stopwords

# ML
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

# Настройка отображения
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✓ Все библиотеки импортированы")

In [ ]:
# Инициализация морфологического анализатора
morph = pymorphy3.MorphAnalyzer()

# Русские стоп-слова
RUSSIAN_STOPWORDS = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все',
    'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по',
    'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему',
    'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть',
    'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом',
    'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для',
    'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз',
    'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому',
    'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем',
    'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно',
    'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот',
    'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три',
    'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть',
    'год', 'года', 'лет', 'также', 'более', 'это', 'день', 'свой', 'который', 'новый',
    'весь', 'время', 'место', 'число', 'дать', 'сказать', 'мочь'
}

print("✓ Инициализация завершена")

## 2. Загрузка данных

In [ ]:
# Загрузка данных из SQLite
db_path = '/content/drive/MyDrive/Colab Notebooks/news_database_final.db'
conn = sqlite3.connect(db_path)

df_sql = pd.read_sql_query("SELECT * FROM news_articles;", conn)
conn.close()

print(f"✓ Загружено из SQLite: {len(df_sql)} записей")
print(f"Колонки: {df_sql.columns.tolist()}")

In [ ]:
# Загрузка данных из JSONL (если файл существует)
jsonl_files = list(Path('/content/drive/MyDrive/gazeta_train.jsonl').glob('*.jsonl'))

df_jsonl_list = []
if jsonl_files:
    print(f"Найдено JSONL файлов: {len(jsonl_files)}")
    for jsonl_file in jsonl_files:
        with open(jsonl_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    if 'root' in data:
                        data = data['root']
                    df_jsonl_list.append(data)
                except:
                    continue

    if df_jsonl_list:
        df_jsonl = pd.DataFrame(df_jsonl_list)
        print(f"✓ Загружено из JSONL: {len(df_jsonl)} записей")
        print(f"Колонки: {df_jsonl.columns.tolist()}")
else:
    df_jsonl = pd.DataFrame()
    print("JSONL файлы не найдены")

In [ ]:
# Объединение данных
if not df_jsonl.empty:
    # Приведение к общему формату
    df_jsonl = df_jsonl.rename(columns={'text': 'description', 'summary': 'summary_text'})

    # Выбираем общие колонки
    common_cols = ['title', 'description', 'url']

    df_sql_subset = df_sql[common_cols].copy()
    df_jsonl_subset = df_jsonl[[col for col in common_cols if col in df_jsonl.columns]].copy()

    # Объединяем
    df = pd.concat([df_sql_subset, df_jsonl_subset], ignore_index=True)
    df = df.drop_duplicates(subset=['url'], keep='first').reset_index(drop=True)
    print(f"✓ Объединено: {len(df)} уникальных записей")
else:
    df = df_sql[['guid', 'title', 'description', 'url', 'published_at', 'source_name']].copy()
    print(f"✓ Используем только SQLite данные: {len(df)} записей")

print(f"\nИтого записей для анализа: {len(df)}")

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Базовая информация
print("=" * 60)
print("БАЗОВАЯ ИНФОРМАЦИЯ О ДАННЫХ")
print("=" * 60)
print(f"Количество записей: {len(df)}")
print(f"Количество признаков: {len(df.columns)}")
print(f"\nТипы данных:")
print(df.dtypes)

print(f"\nПропущенные значения:")
missing = df.isnull().sum()
print(missing[missing > 0])

print(f"\nДубликаты: {df.duplicated().sum()}")

In [ ]:
# Первичный осмотр данных
df.head()

In [ ]:
# Анализ длины текстов
df['text_length'] = df['description'].fillna('').str.len()
df['title_length'] = df['title'].fillna('').str.len()

print("=" * 60)
print("СТАТИСТИКА ДЛИНЫ ТЕКСТОВ")
print("=" * 60)
print("\nОписание (description):")
print(df['text_length'].describe())
print("\nЗаголовок (title):")
print(df['title_length'].describe())

In [ ]:
# Визуализация распределения длины текстов
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df['text_length'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение длины описаний', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Длина текста (символы)')
axes[0].set_ylabel('Частота')
axes[0].axvline(df['text_length'].mean(), color='red', linestyle='--', label=f'Среднее: {df["text_length"].mean():.0f}')
axes[0].legend()

axes[1].hist(df['title_length'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('Распределение длины заголовков', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Длина текста (символы)')
axes[1].set_ylabel('Частота')
axes[1].axvline(df['title_length'].mean(), color='red', linestyle='--', label=f'Среднее: {df["title_length"].mean():.0f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\n📊 Вывод 1: Средняя длина описания новости составляет {df['text_length'].mean():.0f} символов, "
      f"что указывает на информативность текстов для анализа.")

In [ ]:
# Удаление новостей с пустым описанием
df_clean = df[df['text_length'] > 50].copy()
print(f"Удалено {len(df) - len(df_clean)} записей с коротким/пустым описанием")
print(f"Осталось: {len(df_clean)} записей для анализа")

df = df_clean.reset_index(drop=True)

## 4. Предобработка текста

In [ ]:
def preprocess_text(text, lemmatize=True):
    """
    Комплексная предобработка текста:
    - Очистка от HTML
    - Токенизация
    - Лемматизация (опционально)
    - Удаление стоп-слов
    """
    if not isinstance(text, str):
        text = str(text) if text is not None else ""

    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление HTML тегов
    text = re.sub(r'<[^>]+>', '', text)

    # Токенизация (только русские буквы)
    tokens = re.findall(r'[а-яё]+', text)

    # Фильтрация
    result = []
    for token in tokens:
        if len(token) <= 2:  # Удаляем короткие токены
            continue
        if token in RUSSIAN_STOPWORDS:  # Удаляем стоп-слова
            continue

        if lemmatize:
            # Лемматизация
            parsed = morph.parse(token)[0]
            lemma = parsed.normal_form
            result.append(lemma)
        else:
            result.append(token)

    return result

print("Функция предобработки готова")

# Тестирование
sample_text = "В уходящем году инфляция в России находится на историческом минимуме"
print(f"\nПример:\nИсходный текст: {sample_text}")
print(f"Обработанный: {preprocess_text(sample_text)}")

In [ ]:
# Применяем предобработку к заголовкам и описаниям
print("Начало предобработки текстов...")

df['title_tokens'] = df['title'].apply(lambda x: preprocess_text(x, lemmatize=True))
df['desc_tokens'] = df['description'].apply(lambda x: preprocess_text(x, lemmatize=True))

# Объединяем заголовок и описание
df['all_tokens'] = df['title_tokens'] + df['desc_tokens']

# Для TF-IDF нужен текст в виде строки
df['processed_text'] = df['all_tokens'].apply(lambda x: ' '.join(x))

print(f"✓ Предобработка завершена")
print(f"Пример обработанного текста (первые 20 слов):\n{df['all_tokens'].iloc[0][:20]}")

In [ ]:
# Анализ словаря
all_words = []
for tokens in df['all_tokens']:
    all_words.extend(tokens)

word_counts = Counter(all_words)
print(f"\n=" * 60)
print("СТАТИСТИКА СЛОВАРЯ")
print("=" * 60)
print(f"Уникальных слов: {len(word_counts)}")
print(f"Всего слов: {len(all_words)}")
print(f"Средняя длина документа: {len(all_words) / len(df):.1f} слов")

print(f"\nТоп-20 самых частых слов:")
for word, count in word_counts.most_common(20):
    print(f"  {word}: {count}")

print(f"\n📊 Вывод 2: Словарь содержит {len(word_counts)} уникальных слов, "
      f"что свидетельствует о разнообразии тематик новостей.")

In [ ]:
# Облако слов для визуализации
text_for_cloud = ' '.join(all_words)

plt.figure(figsize=(15, 8))
wordcloud = WordCloud(width=1600, height=800,
                     background_color='white',
                     colormap='viridis',
                     max_words=100).generate(text_for_cloud)

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Облако наиболее частых слов в новостях', fontsize=16, fontweight='bold')
plt.tight_layout(pad=0)
plt.show()

print(f"\n📊 Вывод 3: Облако слов показывает преобладание экономической, "
      f"социальной и политической тематик в новостях.")

## 5. Кластеризация слов (формирование тегов)

**Цель**: Найти центры кластеров слов, которые станут тегами для новостей.

In [ ]:
# Создаем словарь: слово -> список документов, в которых оно встречается
word_to_docs = defaultdict(list)
for doc_id, tokens in enumerate(df['all_tokens']):
    for token in set(tokens):  # используем set чтобы не учитывать повторения
        word_to_docs[token].append(doc_id)

# Фильтруем слова: должны встречаться минимум в 3 документах и максимум в 70% документов
min_docs = 3
max_docs = int(len(df) * 0.7)

filtered_words = [word for word, docs in word_to_docs.items()
                 if min_docs <= len(docs) <= max_docs]

print(f"Отфильтровано слов для кластеризации: {len(filtered_words)}")
print(f"(встречаются в {min_docs}-{max_docs} документах)")

In [ ]:
# Создаем матрицу слов: каждое слово представлено вектором документов, в которых оно встречается
# Используем TF-IDF для слов

# Создаем "документы" для слов (каждое слово = документ)
word_documents = []
word_list = []

for word in filtered_words:
    # Для каждого слова создаем контекст из слов, встречающихся в тех же документах
    doc_ids = word_to_docs[word]
    context_words = []

    for doc_id in doc_ids:
        context_words.extend(df['all_tokens'].iloc[doc_id])

    # Удаляем само слово из контекста
    context_words = [w for w in context_words if w != word]

    word_documents.append(' '.join(context_words))
    word_list.append(word)

print(f"Создано {len(word_documents)} 'документов' для слов")

In [ ]:
# Векторизация слов с помощью TF-IDF
word_vectorizer = TfidfVectorizer(max_features=200, min_df=2, max_df=0.8)
word_vectors = word_vectorizer.fit_transform(word_documents)

print(f"Размерность матрицы слов: {word_vectors.shape}")
print(f"({word_vectors.shape[0]} слов × {word_vectors.shape[1]} признаков)")

In [ ]:
# Определяем оптимальное количество кластеров слов с помощью метода локтя
inertias = []
silhouette_scores = []
K_range = range(5, 31, 5)

print("Поиск оптимального количества кластеров для слов...")

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(word_vectors)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(word_vectors, kmeans.labels_))
    print(f"K={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores[-1]:.3f}")

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(K_range, inertias, 'bo-')
axes[0].set_xlabel('Количество кластеров')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Метод локтя для кластеризации слов')
axes[0].grid(True)

axes[1].plot(K_range, silhouette_scores, 'ro-')
axes[1].set_xlabel('Количество кластеров')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score для кластеризации слов')
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Выбираем оптимальное количество кластеров
optimal_k_words = K_range[np.argmax(silhouette_scores)]
print(f"\n✓ Оптимальное количество кластеров слов: {optimal_k_words}")

Поиск оптимального количества кластеров для слов...
K=5: Inertia=10480.86, Silhouette=0.031
K=10: Inertia=9701.99, Silhouette=0.038
K=15: Inertia=9250.36, Silhouette=0.030
K=20: Inertia=8914.24, Silhouette=0.030
K=25: Inertia=8662.76, Silhouette=0.020


In [ ]:
# Кластеризация слов с оптимальным K
kmeans_words = KMeans(n_clusters=optimal_k_words, random_state=42, n_init=10)
word_clusters = kmeans_words.fit_predict(word_vectors)

# Оценка качества кластеризации слов
sil_score_words = silhouette_score(word_vectors, word_clusters)
db_score_words = davies_bouldin_score(word_vectors.toarray(), word_clusters)

print("=" * 60)
print("РЕЗУЛЬТАТЫ КЛАСТЕРИЗАЦИИ СЛОВ")
print("=" * 60)
print(f"Количество кластеров: {optimal_k_words}")
print(f"Silhouette Score: {sil_score_words:.3f}")
print(f"Davies-Bouldin Index: {db_score_words:.3f}")

# Сохраняем кластеры слов
word_cluster_df = pd.DataFrame({
    'word': word_list,
    'cluster': word_clusters
})

print(f"\nРаспределение слов по кластерам:")
print(word_cluster_df['cluster'].value_counts().sort_index())

In [ ]:
# Формируем теги: находим самые центральные слова в каждом кластере
tags_dict = {}

print("=" * 60)
print("ТЕГИ (ЦЕНТРАЛЬНЫЕ СЛОВА КЛАСТЕРОВ)")
print("=" * 60)

for cluster_id in range(optimal_k_words):
    # Получаем слова кластера
    cluster_words = word_cluster_df[word_cluster_df['cluster'] == cluster_id]['word'].tolist()

    # Получаем индексы слов в кластере
    cluster_indices = [i for i, w in enumerate(word_list) if w in cluster_words]

    if not cluster_indices:
        continue

    # Центр кластера
    cluster_center = kmeans_words.cluster_centers_[cluster_id]

    # Находим расстояния от слов до центра
    distances = []
    for idx in cluster_indices:
        dist = np.linalg.norm(word_vectors[idx].toarray() - cluster_center)
        distances.append((word_list[idx], dist))

    # Сортируем по расстоянию и берем 5 самых близких
    closest_words = sorted(distances, key=lambda x: x[1])[:5]
    tags = [word for word, _ in closest_words]

    tags_dict[cluster_id] = tags

    print(f"\nКластер {cluster_id}: {len(cluster_words)} слов")
    print(f"  Теги: {', '.join(tags)}")

print(f"\n📊 Вывод 4: Создано {len(tags_dict)} тематических категорий (тегов) "
      f"для автоматической классификации новостей.")

## 6. Кластеризация новостей

In [ ]:
# Векторизация документов с помощью TF-IDF
doc_vectorizer = TfidfVectorizer(max_features=300, min_df=2, max_df=0.8)
doc_vectors = doc_vectorizer.fit_transform(df['processed_text'])

print(f"Размерность матрицы документов: {doc_vectors.shape}")
print(f"({doc_vectors.shape[0]} документов × {doc_vectors.shape[1]} признаков)")

In [ ]:
# Определяем оптимальное количество кластеров новостей
inertias_docs = []
silhouette_scores_docs = []
K_range_docs = range(3, 16, 2)

print("Поиск оптимального количества кластеров для новостей...")

for k in K_range_docs:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(doc_vectors)
    inertias_docs.append(kmeans.inertia_)
    silhouette_scores_docs.append(silhouette_score(doc_vectors, kmeans.labels_))
    print(f"K={k}: Inertia={kmeans.inertia_:.2f}, Silhouette={silhouette_scores_docs[-1]:.3f}")

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(K_range_docs, inertias_docs, 'bo-')
axes[0].set_xlabel('Количество кластеров')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Метод локтя для кластеризации новостей')
axes[0].grid(True)

axes[1].plot(K_range_docs, silhouette_scores_docs, 'ro-')
axes[1].set_xlabel('Количество кластеров')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette Score для кластеризации новостей')
axes[1].grid(True)

plt.tight_layout()
plt.show()

optimal_k_docs = K_range_docs[np.argmax(silhouette_scores_docs)]
print(f"\n✓ Оптимальное количество кластеров новостей: {optimal_k_docs}")

In [ ]:
# Кластеризация новостей с оптимальным K
kmeans_docs = KMeans(n_clusters=optimal_k_docs, random_state=42, n_init=10)
doc_clusters = kmeans_docs.fit_predict(doc_vectors)

# Добавляем кластеры в dataframe
df['cluster'] = doc_clusters

# Оценка качества кластеризации
sil_score_docs = silhouette_score(doc_vectors, doc_clusters)
db_score_docs = davies_bouldin_score(doc_vectors.toarray(), doc_clusters)

print("=" * 60)
print("РЕЗУЛЬТАТЫ КЛАСТЕРИЗАЦИИ НОВОСТЕЙ")
print("=" * 60)
print(f"Количество кластеров: {optimal_k_docs}")
print(f"Silhouette Score: {sil_score_docs:.3f}")
print(f"Davies-Bouldin Index: {db_score_docs:.3f}")

print(f"\nРаспределение новостей по кластерам:")
cluster_distribution = df['cluster'].value_counts().sort_index()
for cluster_id, count in cluster_distribution.items():
    print(f"  Кластер {cluster_id}: {count} новостей ({count/len(df)*100:.1f}%)")

In [ ]:
# Визуализация кластеров с помощью PCA
pca = PCA(n_components=2, random_state=42)
doc_vectors_2d = pca.fit_transform(doc_vectors.toarray())

plt.figure(figsize=(14, 10))
scatter = plt.scatter(doc_vectors_2d[:, 0], doc_vectors_2d[:, 1],
                     c=doc_clusters, cmap='tab10', s=50, alpha=0.6, edgecolors='black')
plt.colorbar(scatter, label='Cluster ID')
plt.title('Визуализация кластеров новостей (PCA)', fontsize=16, fontweight='bold')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% дисперсии)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% дисперсии)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📊 Вывод 5: Новости успешно разделены на {optimal_k_docs} тематических кластера. "
      f"Визуализация показывает четкую группировку новостей по темам.")

In [ ]:
# Анализ кластеров: топ-слова в каждом кластере
print("=" * 60)
print("ХАРАКТЕРИСТИКА КЛАСТЕРОВ НОВОСТЕЙ")
print("=" * 60)

feature_names = doc_vectorizer.get_feature_names_out()

for cluster_id in range(optimal_k_docs):
    print(f"\n{'='*50}")
    print(f"КЛАСТЕР {cluster_id}: {cluster_distribution[cluster_id]} новостей")
    print(f"{'='*50}")

    # Получаем индексы документов в кластере
    cluster_docs = df[df['cluster'] == cluster_id]

    # Центр кластера
    center = kmeans_docs.cluster_centers_[cluster_id]

    # Топ-10 слов кластера (наибольшие веса)
    top_indices = center.argsort()[-10:][::-1]
    top_words = [feature_names[i] for i in top_indices]

    print(f"Ключевые слова: {', '.join(top_words)}")

    # Примеры заголовков
    print(f"\nПримеры заголовков:")
    for i, title in enumerate(cluster_docs['title'].head(3), 1):
        print(f"  {i}. {title}")

## 7. Присвоение тегов новостям

Используем кластеры слов для автоматического тегирования новостей.

In [ ]:
def assign_tags_to_news(news_tokens, word_cluster_df, tags_dict, top_n=3):
    """
    Присваивает теги новости на основе кластеров слов.

    Args:
        news_tokens: список токенов новости
        word_cluster_df: DataFrame с кластерами слов
        tags_dict: словарь {cluster_id: [tags]}
        top_n: количество тегов для присвоения

    Returns:
        список тегов
    """
    # Считаем, сколько раз встречаются слова из каждого кластера
    cluster_counts = Counter()

    for token in news_tokens:
        word_info = word_cluster_df[word_cluster_df['word'] == token]
        if not word_info.empty:
            cluster_id = word_info['cluster'].iloc[0]
            cluster_counts[cluster_id] += 1

    # Выбираем топ-N кластеров
    top_clusters = cluster_counts.most_common(top_n)

    # Формируем теги
    assigned_tags = []
    for cluster_id, _ in top_clusters:
        if cluster_id in tags_dict:
            assigned_tags.extend(tags_dict[cluster_id][:2])  # берем 2 тега из кластера

    return assigned_tags[:top_n*2]  # максимум top_n*2 тегов

# Применяем функцию к каждой новости
df['tags'] = df['all_tokens'].apply(
    lambda x: assign_tags_to_news(x, word_cluster_df, tags_dict, top_n=3)
)

print("✓ Теги присвоены новостям")
print(f"\nПримеры новостей с тегами:\n")
for i in range(min(5, len(df))):
    print(f"Заголовок: {df['title'].iloc[i][:80]}...")
    print(f"Теги: {', '.join(df['tags'].iloc[i])}")
    print(f"Кластер: {df['cluster'].iloc[i]}")
    print()

In [ ]:
# Статистика по тегам
all_tags = []
for tags in df['tags']:
    all_tags.extend(tags)

tag_counts = Counter(all_tags)

print("=" * 60)
print("СТАТИСТИКА ПО ТЕГАМ")
print("=" * 60)
print(f"Всего уникальных тегов использовано: {len(tag_counts)}")
print(f"Среднее количество тегов на новость: {len(all_tags) / len(df):.1f}")

print(f"\nТоп-20 самых частых тегов:")
for tag, count in tag_counts.most_common(20):
    print(f"  {tag}: {count} раз")

# Визуализация топ-тегов
top_tags = dict(tag_counts.most_common(15))

plt.figure(figsize=(12, 6))
plt.barh(list(top_tags.keys()), list(top_tags.values()), color='steelblue')
plt.xlabel('Частота использования')
plt.title('Топ-15 самых частых тегов', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 8. Модель 1 (Baseline): Логистическая регрессия

Обучаем простую модель для классификации новостей по кластерам.

In [ ]:
# Подготовка данных для supervised learning
X = doc_vectors  # TF-IDF векторы
y = df['cluster']  # Кластеры как целевая переменная

# Разбиение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")
print(f"\nРаспределение классов в обучающей выборке:")
print(y_train.value_counts().sort_index())

In [ ]:
# Baseline модель: Логистическая регрессия
print("Обучение Baseline модели (Logistic Regression)...")

baseline_model = LogisticRegression(max_iter=1000, random_state=42)
baseline_model.fit(X_train, y_train)

# Предсказания
y_pred_baseline = baseline_model.predict(X_test)

# Оценка качества
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
f1_baseline = f1_score(y_test, y_pred_baseline, average='weighted')
precision_baseline = precision_score(y_test, y_pred_baseline, average='weighted')
recall_baseline = recall_score(y_test, y_pred_baseline, average='weighted')

print("=" * 60)
print("РЕЗУЛЬТАТЫ BASELINE МОДЕЛИ (Logistic Regression)")
print("=" * 60)
print(f"Accuracy:  {accuracy_baseline:.4f}")
print(f"Precision: {precision_baseline:.4f}")
print(f"Recall:    {recall_baseline:.4f}")
print(f"F1-score:  {f1_baseline:.4f}")

print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_baseline))

In [ ]:
# Confusion Matrix для baseline
cm_baseline = confusion_matrix(y_test, y_pred_baseline)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues',
           xticklabels=range(optimal_k_docs),
           yticklabels=range(optimal_k_docs))
plt.title('Confusion Matrix - Baseline (Logistic Regression)', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 9. Модель 2 (Улучшенная): Random Forest с GridSearchCV

In [ ]:
# Random Forest с подбором гиперпараметров
print("Обучение улучшенной модели (Random Forest с GridSearchCV)...")

rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\n✓ Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший CV F1-score: {grid_search.best_score_:.4f}")

In [ ]:
# Оценка на тестовой выборке
best_rf = grid_search.best_estimator_
y_pred_rf = best_rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')

print("=" * 60)
print("РЕЗУЛЬТАТЫ УЛУЧШЕННОЙ МОДЕЛИ (Random Forest)")
print("=" * 60)
print(f"Accuracy:  {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall:    {recall_rf:.4f}")
print(f"F1-score:  {f1_rf:.4f}")

print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Confusion Matrix для Random Forest
cm_rf = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens',
           xticklabels=range(optimal_k_docs),
           yticklabels=range(optimal_k_docs))
plt.title('Confusion Matrix - Random Forest', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 10. Сравнение моделей

In [ ]:
# Сравнительная таблица
results_comparison = pd.DataFrame({
    'Модель': ['Baseline (Logistic Regression)', 'Random Forest (GridSearchCV)'],
    'Accuracy': [accuracy_baseline, accuracy_rf],
    'Precision': [precision_baseline, precision_rf],
    'Recall': [recall_baseline, recall_rf],
    'F1-score': [f1_baseline, f1_rf],
    'CV Score': ['-', grid_search.best_score_]
})

print("=" * 80)
print("СРАВНЕНИЕ МОДЕЛЕЙ")
print("=" * 80)
print(results_comparison.to_string(index=False))

# Визуализация сравнения
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-score']
baseline_scores = [accuracy_baseline, precision_baseline, recall_baseline, f1_baseline]
rf_scores = [accuracy_rf, precision_rf, recall_rf, f1_rf]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, baseline_scores, width, label='Baseline (Logistic Reg.)', color='skyblue')
bars2 = ax.bar(x + width/2, rf_scores, width, label='Random Forest', color='lightgreen')

ax.set_ylabel('Score')
ax.set_title('Сравнение метрик моделей', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylim([0.7, 1.0])
ax.grid(True, alpha=0.3, axis='y')

# Добавляем значения на столбцы
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',
                   xy=(bar.get_x() + bar.get_width() / 2, height),
                   xytext=(0, 3),
                   textcoords="offset points",
                   ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

improvement = (f1_rf - f1_baseline) / f1_baseline * 100
print(f"\n📊 Вывод 6: Random Forest показывает улучшение F1-score на {improvement:.1f}% "
      f"по сравнению с baseline моделью.")

## 11. Анализ ошибок

In [ ]:
# Находим неправильно классифицированные примеры
test_indices = X_test.indices if hasattr(X_test, 'indices') else range(len(y_test))
df_test = df.iloc[list(X_test.indices) if hasattr(X_test, 'indices') else y_test.index].copy()
df_test['predicted'] = y_pred_rf
df_test['actual'] = y_test.values

# Находим ошибки
errors = df_test[df_test['predicted'] != df_test['actual']].copy()

print("=" * 60)
print("АНАЛИЗ ОШИБОК")
print("=" * 60)
print(f"Всего ошибок: {len(errors)}")
print(f"Процент ошибок: {len(errors) / len(df_test) * 100:.2f}%")

print(f"\nПримеры ошибочных классификаций:\n")
for i, (idx, row) in enumerate(errors.head(5).iterrows(), 1):
    print(f"\n--- Пример {i} ---")
    print(f"Заголовок: {row['title'][:100]}...")
    print(f"Реальный кластер: {row['actual']}")
    print(f"Предсказанный кластер: {row['predicted']}")
    print(f"Теги: {', '.join(row['tags'][:5])}")

In [ ]:
# Анализ наиболее часто путающихся пар кластеров
error_pairs = Counter()
for _, row in errors.iterrows():
    pair = (row['actual'], row['predicted'])
    error_pairs[pair] += 1

print("\nНаиболее часто путающиеся пары кластеров:")
for (actual, predicted), count in error_pairs.most_common(5):
    print(f"  Кластер {actual} → Кластер {predicted}: {count} ошибок")

## 12. Feature Importance

In [ ]:
# Анализ важности признаков для Random Forest
feature_importance = best_rf.feature_importances_
feature_names = doc_vectorizer.get_feature_names_out()

# Топ-20 самых важных признаков
indices = np.argsort(feature_importance)[-20:]
top_features = [(feature_names[i], feature_importance[i]) for i in indices]

print("=" * 60)
print("ТОП-20 САМЫХ ВАЖНЫХ ПРИЗНАКОВ (слов)")
print("=" * 60)
for feature, importance in reversed(top_features):
    print(f"  {feature}: {importance:.4f}")

# Визуализация
plt.figure(figsize=(10, 8))
plt.barh([f[0] for f in top_features], [f[1] for f in top_features], color='coral')
plt.xlabel('Importance')
plt.title('Топ-20 самых важных слов для классификации', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n📊 Вывод 7: Модель успешно выявила наиболее значимые слова, "
      f"которые помогают различать тематики новостей.")

## 13. Итоговые выводы

In [ ]:
print("=" * 80)
print("ИТОГОВЫЕ ВЫВОДЫ ПРОЕКТА")
print("=" * 80)

print(f"""
1. ДАННЫЕ:
   - Проанализировано {len(df)} новостей
   - Средняя длина новости: {df['text_length'].mean():.0f} символов
   - Словарь: {len(word_counts)} уникальных слов

2. КЛАСТЕРИЗАЦИЯ СЛОВ:
   - Создано {optimal_k_words} кластеров слов (тегов)
   - Silhouette Score: {sil_score_words:.3f}
   - Davies-Bouldin Index: {db_score_words:.3f}

3. КЛАСТЕРИЗАЦИЯ НОВОСТЕЙ:
   - Новости разделены на {optimal_k_docs} тематических кластера
   - Silhouette Score: {sil_score_docs:.3f}
   - Davies-Bouldin Index: {db_score_docs:.3f}

4. МОДЕЛИ КЛАССИФИКАЦИИ:

   Baseline (Logistic Regression):
   - Accuracy: {accuracy_baseline:.4f}
   - F1-score: {f1_baseline:.4f}

   Улучшенная модель (Random Forest):
   - Accuracy: {accuracy_rf:.4f}
   - F1-score: {f1_rf:.4f}
   - Улучшение: +{(f1_rf - f1_baseline)*100:.2f}%

5. КЛЮЧЕВЫЕ ДОСТИЖЕНИЯ:
   - ✓ Автоматическое тегирование новостей работает успешно
   - ✓ Random Forest показывает высокую точность классификации ({accuracy_rf:.1%})
   - ✓ Система может быть использована для автоматической организации новостного контента
   - ✓ Выявлены наиболее информативные слова для каждой темы

6. ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:
   - Автоматическая категоризация новостей
   - Рекомендательная система
   - Поиск похожих новостей
   - Мониторинг тематик в СМИ
""")

print("=" * 80)

## 14. Сохранение результатов

In [ ]:
# Сохраняем результаты
import pickle

# Модели
with open('/mnt/user-data/outputs/best_rf_model.pkl', 'wb') as f:
    pickle.dump(best_rf, f)

with open('/mnt/user-data/outputs/baseline_model.pkl', 'wb') as f:
    pickle.dump(baseline_model, f)

# Векторизатор
with open('/mnt/user-data/outputs/vectorizer.pkl', 'wb') as f:
    pickle.dump(doc_vectorizer, f)

# Результаты
df[['title', 'cluster', 'tags']].to_csv('/mnt/user-data/outputs/news_with_clusters_and_tags.csv',
                                         index=False, encoding='utf-8')

word_cluster_df.to_csv('/mnt/user-data/outputs/word_clusters.csv', index=False, encoding='utf-8')

# Сохраняем теги
with open('/mnt/user-data/outputs/tags_dictionary.json', 'w', encoding='utf-8') as f:
    json.dump({str(k): v for k, v in tags_dict.items()}, f, ensure_ascii=False, indent=2)

print("✓ Результаты сохранены:")
print("  - best_rf_model.pkl")
print("  - baseline_model.pkl")
print("  - vectorizer.pkl")
print("  - news_with_clusters_and_tags.csv")
print("  - word_clusters.csv")
print("  - tags_dictionary.json")